In [62]:
import pandas as pd
import requests
from my_keys import password_sql
from my_keys import api_weather_key
from my_keys import api_aerodata_key

In [64]:
from my_keys import password_sql
schema = "cities_cloud"
host = "104.155.47.10"
user = "root"
password = password_sql
port = 3306
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [66]:
cities_from_sql = pd.read_sql("SELECT * FROM cities", con=connection_string)
cities_from_sql

,city_id,City,Country,Latitude,Longitude
0,1,Berlin,Germany,52.520000,13.405000
1,2,Hamburg,Germany,53.550000,10.000000
2,3,Munich,Germany,48.137500,11.575000
3,4,Paris,France,48.856667,2.352222


In [40]:
#Weather data full function

import pandas as pd
import requests
from pytz import timezone
from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Step 0 Clear database: Create the database connection string

def retrieve_and_send_data():
    
    # Step 1: Create the database connection string
    connection_string = create_connection_string()
    
    # Step 2: Fetch city data from the 'cities' table
    cities_df = fetch_cities_data(connection_string)
    
    # Step 3: Fetch weather data for each city
    weather_df = fetch_weather_data(cities_df)

    # Step 4: Clear table and store the weather data in the 'weather' table
    send_weather_data(weather_df, connection_string)
    
    return("Data has been updated")

def create_connection_string():
    schema = "city_data"
    host = "127.0.0.1"
    user = "root"
    password = password_sql  # Replace with actual password
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
    # Fetch city data from the 'cities' table
    return pd.read_sql("SELECT city_id, latitude, longitude FROM cities", con=connection_string)

def fetch_weather_data(cities_df):
    berlin_timezone = timezone('Europe/Berlin')
    API_key = api_weather_key  # Replace with your actual OpenWeather API key
    weather_items = []

    for _, city in cities_df.iterrows():
        city_id = city["city_id"]
        latitude = city["latitude"]
        longitude = city["longitude"]

        # Fetch weather forecast data from the API
        url = f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric"
        response = requests.get(url)
        weather_data = response.json()

        # **Add this error check**
        if "list" not in weather_data:
            print(f"Error: Missing 'list' data for city_id {city_id}. API response: {weather_data}")
            continue  # Skip to the next city if there's no forecast data

        # Record the retrieval time in Berlin timezone
        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        # Loop through the forecast data and extract relevant fields
        for item in weather_data["list"]:
            weather_item = {
                "city_id": city_id,
                "forecast_time": item.get("dt_txt"),  # Forecast timestamp
                "temperature": item["main"].get("temp"),  # Temperature in Celsius
                "forecast": item["weather"][0].get("main"),  # Weather condition
                "rain_in_last_3h": item.get("rain", {}).get("3h", 0),  # Rain volume
                "wind_speed": item["wind"].get("speed"),  # Wind speed
                "data_retrieved_at": retrieval_time  # Time data was retrieved
            }
            weather_items.append(weather_item)

    # Convert list of weather data into a DataFrame
    weather_df = pd.DataFrame(weather_items)
    
    # Ensure datetime columns are correctly formatted
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"], errors='coerce')
    weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"], errors='coerce')

    return weather_df

def send_weather_data(weather_df, connection_string):
    engine = create_engine(connection_string)
    try:
        # Using a transaction block to ensure that the operation is atomic*
        with engine.begin() as connection:
            # Using the text construct to ensure the SQL command is executed as raw SQL (otherwise won't work)
            connection.execute(text("TRUNCATE TABLE weather"))
            weather_df.to_sql('weather', 
                              if_exists='append', 
                              con=connection, 
                              index=False)
    except SQLAlchemyError as e:
        print(f"Error occurred: {e}")


# Run the entire data fetching and storing process
retrieve_and_send_data()

'Data has been updated'

In [41]:
weather_from_sql = pd.read_sql("SELECT * FROM weather", con=connection_string)
print(weather_from_sql)

     city_id       forecast_time  temperature forecast  rain_in_last_3h  \
0          1 2024-11-06 15:00:00         6.22   Clouds             0.00   
1          1 2024-11-06 18:00:00         6.83   Clouds             0.00   
2          1 2024-11-06 21:00:00         6.80   Clouds             0.00   
3          1 2024-11-07 00:00:00         6.38   Clouds             0.00   
4          1 2024-11-07 03:00:00         5.84   Clouds             0.00   
..       ...                 ...          ...      ...              ...   
155        4 2024-11-11 00:00:00        14.32     Rain             0.15   
156        4 2024-11-11 03:00:00        14.02   Clouds             0.00   
157        4 2024-11-11 06:00:00        13.60     Rain             0.12   
158        4 2024-11-11 09:00:00        14.09   Clouds             0.00   
159        4 2024-11-11 12:00:00        15.98   Clouds             0.00   

     wind_speed   data_retrieved_at  
0          0.39 2024-11-06 15:52:43  
1          0.40 2024-11

In [70]:
airports_from_sql = pd.read_sql("SELECT * FROM airports", con=connection_string)
print(airports_from_sql)

   city_id                         airport_name icao_code    data_retrieved_at
0      1.0                   Berlin Brandenburg      EDDB  2024-11-07 12:15:43
1      2.0                             Hamburg       EDDH  2024-11-07 12:15:44
2      2.0                              Bremen       EDDW  2024-11-07 12:15:44
3      3.0                              Munich       EDDM  2024-11-07 12:15:44
4      3.0                           Innsbruck       LOWI  2024-11-07 12:15:44
5      4.0                    Paris -Le Bourget      LFPB  2024-11-07 12:15:44
6      4.0                          Paris -Orly      LFPO  2024-11-07 12:15:44
7      4.0              Paris Charles de Gaulle      LFPG  2024-11-07 12:15:44
8      4.0  Beauvais/Tillé Paris Beauvais Tillé      LFOB  2024-11-07 12:15:44


In [72]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
from sqlalchemy import create_engine

def get_flight_data_and_store():
    # Set up database connection parameters
    schema = "city_data"
    host = "127.0.0.1"
    user = "root"
    password = password_sql
    connection_string = f'mysql+pymysql://{user}:{password}@{host}/{schema}'
    engine = create_engine(connection_string)

    # Fetch ICAO codes from the airports table
    airports_df = pd.read_sql("SELECT icao_code FROM airports", con=engine)
    icao_list = airports_df["icao_code"].tolist()

    # API key for Aerodatabox
    api_key = api_aerodata_key
    berlin_timezone = timezone('Europe/Berlin')
    tomorrow = (datetime.now(berlin_timezone) + timedelta(days=1))

    flight_items = []

    for icao in icao_list:
        # Define time ranges for the API calls
        times = [["00:00", "11:59"], ["12:00", "23:59"]]

        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow.strftime('%Y-%m-%d')}T{time[0]}/{tomorrow.strftime('%Y-%m-%d')}T{time[1]}"
            querystring = {"withLeg":"true",
                    "direction":"Arrival",
                    "withCancelled":"false",
                    "withCodeshared":"true",
                    "withCargo":"false",
                    "withPrivate":"false"}

            headers = {
                'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
                'x-rapidapi-key': api_key
            }

            # Make the API request
            response = requests.get(url, headers=headers, params=querystring)

            # Check if the response was successful
            if response.status_code != 200:
                print(f"Error fetching data for {icao}: {response.status_code} - {response.text}")
                continue  # Skip to the next ICAO if there was an error

            try:
                flights_json = response.json()
            except ValueError as e:
                print(f"Error decoding JSON for {icao}: {e}")
                print("Response text:", response.text)  # Print the raw response text
                continue  # Skip to the next ICAO if JSON decoding fails

            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            arrivals = flights_json.get("arrivals", [])
            if not arrivals:
                print(f"No arrivals found for ICAO: {icao}")
                continue  # Skip to the next ICAO if no arrivals

            for item in arrivals:
                flight_item = {
                    "arrival_airport_icao": icao,
                    "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                    "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                    "flight_number": item.get("number", None),
                    "data_retrieved_at": retrieval_time
                }
                flight_items.append(flight_item)

    # Create a DataFrame from the collected flight items
    flights_df = pd.DataFrame(flight_items)

    # Check if the DataFrame is empty before processing
    if not flights_df.empty:
        flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
        flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
        flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])

        # Write the DataFrame to the flights table in the database
        flights_df.to_sql('flights', con=engine, if_exists='append', index=False)
        print("Flight data stored successfully.")
    else:
        print("No flight data available to store.")

# Call the function to execute the process
get_flight_data_and_store()

Error fetching data for LFPB: 204 - 
Error fetching data for LFPB: 204 - 
Flight data stored successfully.


In [58]:
flights_from_sql = pd.read_sql("SELECT * FROM flights", con=connection_string)
print(flights_from_sql)

      flight_id arrival_airport_icao departure_airport_icao  \
0             1                 EDDB                   OTHH   
1             2                 EDDB                   LIMC   
2             3                 EDDB                   LBSF   
3             4                 EDDB                   EDDS   
4             5                 EDDB                   LFSB   
...         ...                  ...                    ...   
8730       8731                 LOWI                   EDDF   
8731       8732                 LOWI                   LOWW   
8732       8733                 LOWI                   EDDF   
8733       8734                 LOWI                   EDDF   
8734       8735                 LOWI                   LOWW   

     scheduled_arrival_time flight_number   data_retrieved_at  
0       2024-11-07 06:55:00         QR 79 2024-11-06 12:14:58  
1       2024-11-07 07:30:00          FR 9 2024-11-06 12:14:58  
2       2024-11-07 07:30:00       FR 1149 2024-11-0